In [2]:
from resinkit import AsyncResinkitClient
import asyncio
import logging

# Configure the root logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logging.getLogger("httpcore.http11").setLevel(logging.INFO)
logging.getLogger('httpcore.connection').setLevel(logging.INFO)

import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops

rc = AsyncResinkitClient(flink_gateway_url="http://localhost:8083")
res = asyncio.run(rc.query('''
SELECT * FROM (
    VALUES
        (23, 'Alice Liddell', CAST('2024-12-29 10:30:00' AS TIMESTAMP)),
        (19, 'Bob Smith', CAST('2024-12-28 11:45:00' AS TIMESTAMP)),
        (27, 'Charlie Brown', CAST('2024-12-27 14:15:00' AS TIMESTAMP)),
        (31, 'David Jones', CAST('2024-12-26 16:20:00' AS TIMESTAMP))
) AS t(age, name, created_at);
''', n_row_limit=3))

res

2024-12-29 19:54:00,163 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions "HTTP/1.1 200 OK"
2024-12-29 19:54:00,164 - resinkit.async_resinkit_client - DEBUG - Opened session: 48288e13-2f58-43a2-9996-f364f997146c
2024-12-29 19:54:00,168 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/48288e13-2f58-43a2-9996-f364f997146c/statements "HTTP/1.1 200 OK"
2024-12-29 19:54:00,168 - resinkit.async_resinkit_client - DEBUG - Select result: {'operationHandle': '08c3a387-a9eb-4d6a-8f51-dff209221ff9'}
2024-12-29 19:54:00,173 - httpx - INFO - HTTP Request: GET http://localhost:8083/sessions/48288e13-2f58-43a2-9996-f364f997146c/operations/08c3a387-a9eb-4d6a-8f51-dff209221ff9/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-29 19:54:00,173 - resinkit.async_resinkit_client - DEBUG - Fetch result: {
  "nextResultUri": "/v2/sessions/48288e13-2f58-43a2-9996-f364f997146c/operations/08c3a387-a9eb-4d6a-8f51-dff209221ff9/result/0?rowFormat=JSON",
  "resultType": "NOT_READY"


,age,name,created_at
0,23,Alice Liddell,2024-12-29T10:30:00
1,19,Bob Smith,2024-12-28T11:45:00
2,27,Charlie Brown,2024-12-27T14:15:00


In [5]:
# languange=sql
sql1 = '''
CREATE TABLE mydatabase_User (
  `event_time` TIMESTAMP_LTZ(3) METADATA FROM 'value.source.timestamp' VIRTUAL,  -- from Debezium format
  `origin_table` STRING METADATA FROM 'value.source.table' VIRTUAL, -- from Debezium format
  `partition_id` BIGINT METADATA FROM 'partition' VIRTUAL,  -- from Kafka connector
  `offset` BIGINT METADATA VIRTUAL,  -- from Kafka connector
  `id` STRING,
  `name` STRING,
  `email` STRING,
  `email_verified` TIMESTAMP_LTZ(3),
  `image` STRING,
  `invalid_login_attempts` INT,
  `locked_at` TIMESTAMP_LTZ(3),
  `password` STRING,
  `created_at` TIMESTAMP_LTZ(3),
  `updated_at` TIMESTAMP_LTZ(3)
) WITH (
    'connector' = 'kafka',
    'topic' = 'mydatabase.User',
    'properties.bootstrap.servers' = 'localhost:9092',
    'properties.group.id' = 'mydatabase_user_consumer_group',
    'scan.startup.mode' = 'earliest-offset',
    'format' = 'debezium-json'
);
'''
sql2 = """ ADD JAR '/opt/flink/lib/flink-sql-connector-kafka-3.3.0-1.19.jar' """
sql3 = 'select * from mydatabase_User;'

res2 = asyncio.run(rc.query_multiple([sql1, sql2, sql3], n_row_limit=10, max_poll_secs=10))
res2


2024-12-29 20:01:12,478 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions "HTTP/1.1 200 OK"
2024-12-29 20:01:12,479 - resinkit.async_resinkit_client - DEBUG - Opened session for multiple queries: f8f48de3-591c-467f-b4f0-1c322f19b682
2024-12-29 20:01:12,480 - resinkit.async_resinkit_client - DEBUG - Running query: request: {
  "statement": "\nCREATE TABLE mydatabase_User (\n  `event_time` TIMESTAMP_LTZ(3) METADATA FROM 'value.source.timestamp' VIRTUAL,  -- from Debezium format\n  `origin_table` STRING METADATA FROM 'value.source.table' VIRTUAL, -- from Debezium format\n  `partition_id` BIGINT METADATA FROM 'partition' VIRTUAL,  -- from Kafka connector\n  `offset` BIGINT METADATA VIRTUAL,  -- from Kafka connector\n  `id` STRING,\n  `name` STRING,\n  `email` STRING,\n  `email_verified` TIMESTAMP_LTZ(3),\n  `image` STRING,\n  `invalid_login_attempts` INT,\n  `locked_at` TIMESTAMP_LTZ(3),\n  `password` STRING,\n  `created_at` TIMESTAMP_LTZ(3),\n  `updated_at` TIMESTAMP_LTZ(

[  result
 0     OK,
   result
 0     OK,
   event_time origin_table  partition_id  offset  \
 0       None         User             0       0   
 1       None         User             0       1   
 2       None         User             0       2   
 3       None         User             0       3   
 4       None         User             0       4   
 5       None         User             0       5   
 6       None         User             0       6   
 7       None         User             0       7   
 8       None         User             0       8   
 9       None         User             0       9   
 
                                      id             name  \
 0  b366f235-7991-4fdf-a8fe-600843b6ebf9     Paula Chavez   
 1  76720e39-8bd1-45d1-9568-ff0923ddd9e8  Amber Walker MD   
 2  699522bc-4487-41cc-9752-0167d0f30800      Kelsey Mata   
 3  9679da90-7062-4dba-8451-c8839ce571ab     James Miller   
 4  b52e2329-81a4-4075-bac6-0308216bbaa2      Rebecca Lee   
 5  a1281172-395a-

In [6]:
res2

[  result
 0     OK,
   result
 0     OK,
   event_time origin_table  partition_id  offset  \
 0       None         User             0       0   
 1       None         User             0       1   
 2       None         User             0       2   
 3       None         User             0       3   
 4       None         User             0       4   
 5       None         User             0       5   
 6       None         User             0       6   
 7       None         User             0       7   
 8       None         User             0       8   
 9       None         User             0       9   
 
                                      id             name  \
 0  b366f235-7991-4fdf-a8fe-600843b6ebf9     Paula Chavez   
 1  76720e39-8bd1-45d1-9568-ff0923ddd9e8  Amber Walker MD   
 2  699522bc-4487-41cc-9752-0167d0f30800      Kelsey Mata   
 3  9679da90-7062-4dba-8451-c8839ce571ab     James Miller   
 4  b52e2329-81a4-4075-bac6-0308216bbaa2      Rebecca Lee   
 5  a1281172-395a-